# Decadal Calendar Fix

In [1]:
import xarray as xr

In [2]:
ds = xr.open_dataset("/Users/pingu/Desktop/69de4961fa0f43f54a40b106b808959d/psl_Amon_EC-Earth3_dcppB-forecast_r10i1p1f1_gr_20201116-20201216.nc")
ds

<xarray.Dataset>
Dimensions:      (time: 2, realization: 10, bnds: 2, lat: 256, lon: 512)
Coordinates:
  * time         (time) datetime64[ns] 2020-11-16 2020-12-16T12:00:00
  * lat          (lat) float64 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
  * lon          (lon) float64 0.3516 1.055 1.758 2.461 ... 358.2 358.9 359.6
    reftime      datetime64[ns] ...
    leadtime     (time) timedelta64[ns] ...
  * realization  (realization) int32 10 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: bnds
Data variables:
    time_bnds    (realization, time, bnds) datetime64[ns] ...
    lat_bnds     (realization, time, lat, bnds) float64 ...
    lon_bnds     (realization, time, lon, bnds) float64 ...
    psl          (realization, time, lat, lon) float32 ...
Attributes: (12/52)
    Conventions:                 CF-1.7 CMIP-6.2
    activity_id:                 DCPP
    branch_method:               no parent
    branch_time:                 0.0
    branch_time_in_child:        0.0
    branch_time_in_parent:       0.0
    ...                          ...
    license:                     CMIP6 model data produced by EC-Earth-Consor...
    cmor_version:                3.4.0
    forcing_description:         f1, CMIP6 historical forcings
    physics_description:         physics from the standard model configuratio...
    initialization_description:  Atmosphere initialization based on full-fiel...
    startdate:                   s202011

In [3]:
ds.time.encoding.get("calendar")

'proleptic_gregorian'

## apply calendar fix

In [4]:
from rook.utils import decadal_fixes

In [5]:
ds_fix = decadal_fixes.decadal_fix_calendar(None, ds)
ds_fix

<xarray.Dataset>
Dimensions:      (time: 2, realization: 10, bnds: 2, lat: 256, lon: 512)
Coordinates:
  * time         (time) datetime64[ns] 2020-11-16 2020-12-16T12:00:00
  * lat          (lat) float64 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
  * lon          (lon) float64 0.3516 1.055 1.758 2.461 ... 358.2 358.9 359.6
    reftime      datetime64[ns] ...
    leadtime     (time) timedelta64[ns] ...
  * realization  (realization) int32 10 1 2 3 4 5 6 7 8 9
Dimensions without coordinates: bnds
Data variables:
    time_bnds    (realization, time, bnds) datetime64[ns] ...
    lat_bnds     (realization, time, lat, bnds) float64 ...
    lon_bnds     (realization, time, lon, bnds) float64 ...
    psl          (realization, time, lat, lon) float32 ...
Attributes: (12/52)
    Conventions:                 CF-1.7 CMIP-6.2
    activity_id:                 DCPP
    branch_method:               no parent
    branch_time:                 0.0
    branch_time_in_child:        0.0
    branch_time_in_parent:       0.0
    ...                          ...
    license:                     CMIP6 model data produced by EC-Earth-Consor...
    cmor_version:                3.4.0
    forcing_description:         f1, CMIP6 historical forcings
    physics_description:         physics from the standard model configuratio...
    initialization_description:  Atmosphere initialization based on full-fiel...
    startdate:                   s202011

In [6]:
ds_fix.time.encoding.get("calendar")

'standard'